# Roocs data stores

Roocs consists of stores which it uses to identify and apply fixes to datasets. This notebook demonstrates the purpose of and how to interact with the Character store and the Fix store.

The Character store contains information about the 'character' of datasets that have been scanned. This is used in the identification of fixes.
    
The Fix store keeps track of these fixes. Each document corresponds to a specific dataset and details the fixes which are to be applied to it before an operation is applied to it.

In [ ]:
DIAGRAM

## Working with the character store

In [3]:
# Import the required packages
import virtualenv
import pip
import os

# Define and create the base directory install virtual environments
venvs_dir = os.path.join(os.path.expanduser("~"), "nb-venvs")

if not os.path.isdir(venvs_dir):
    os.makedirs(venvs_dir)
    
# Define the venv directory
venv_dir = os.path.join(venvs_dir, 'venv-dachar-notebook')

In [4]:
# Create the virtual environment
virtualenv.create_environment(venv_dir)

copying /opt/jaspy/bin/python => /home/users/esmith88/nb-venvs/venv-dachar-notebook/bin/python


In [5]:
# Activate the venv
activate_file = os.path.join(venv_dir, "bin", "activate_this.py")
exec(open(activate_file).read(), dict(__file__=activate_file))

In [ ]:
# pip install a package using the venv as a prefix
pip.main(["install", "--prefix", venv_dir, "xarray>=0.15"])
pip.main(["install", "--prefix", venv_dir, "netcdftime"])
#pip.main(["install", "--prefix",venv_dir, "-e", "git+https://github.com/roocs/dachar.git@dev-elle#egg=dachar"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Using cached xarray-0.16.0-py3-none-any.whl (704 kB)
  Using cached pandas-1.0.5-cp37-cp37m-manylinux1_x86_64.whl (10.1 MB)
  Using cached setuptools-49.2.0-py3-none-any.whl (789 kB)


ERROR: scitools-iris 2.2.0 has requirement matplotlib<3,>=2, but you'll have matplotlib 3.0.2 which is incompatible.


  Attempting uninstall: pandas
    Found existing installation: pandas 0.23.4


    Not uninstalling pandas at /opt/jaspy/lib/python3.7/site-packages, outside environment /home/users/esmith88/nb-venvs/venv-dachar-notebook
    Can't uninstall 'pandas'. No files were found to uninstall.


In [17]:
from dachar.dachar.utils import get_stores
char_store = get_stores.get_dc_store('elasticsearch')
char_store.get_all()

ModuleNotFoundError: No module named 'dachar'